# WorkFlow 1: Load WebSite content into Vector DB

1.   Use LangChain LCEL
2.  Prompting + LCEL + Output Parser
3.  RAG (build once, re-use) with sources




In [ ]:
#Map Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
#%pip install --upgrade jupyter-client

In [ ]:
%pip install -qU \
    "requests" \
    "langchain" \
    "langchain-openai" \
    "langchain-community" \
    "langchain-text-splitters" \
    beautifulsoup4 lxml faiss-cpu langchainhub tavily-python "gradio"

In [ ]:
import importlib
def _ver(name):
    try:
        m = importlib.import_module(name)
        return getattr(m, "__version__", "n/a")
    except Exception as e:
        return f"not installed ({e})"
print("langchain           :", _ver("langchain"))
print("langgraph           :", _ver("langgraph"))
print("langchain-core      :", _ver("langchain_core"))
print("langchain-community :", _ver("langchain_community"))
print("langchain-openai    :", _ver("langchain_openai"))
print("langchainhub        :", _ver("langchainhub"))
print("langchain-text-splitters:", _ver("langchain_text_splitters"))
print("faiss-cpu           :", _ver("faiss"))
print("tavily-python       :", _ver("tavily"))

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Load URL content

In [ ]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
import bs4
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser


# 1) Load docs (pick any public pages you want indexed)
urls = [
    "https://www.apple.com/",
    "https://www.apple.com/iphone/",
  # "https://www.apple.com/ipad/",
  # "https://www.apple.com/watch/",
    "https://www.apple.com/mac/"
]

In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

loader = WebBaseLoader(web_paths=urls, bs_kwargs=dict(
                parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header")
        )
      ),
    )
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,
                                               chunk_overlap=200)
splits = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

vectorstore = FAISS.from_documents(docs, embeddings)
retriever = vectorstore.as_retriever()


In [ ]:
from langchain_classic.chains import create_history_aware_retriever  # as of 20251214, using classic -> this is an open issue with standard langchain library https://github.com/langchain-ai/langchain-community/issues/433
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", contextualize_q_system_prompt),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}"),
])

# 4. Create the history-aware retriever chain
# This chain takes the current input and chat history, generates a standalone question,
# and passes it to the base retriever to fetch relevant documents.
history_aware_retriever = create_history_aware_retriever(
    llm=llm,
    retriever=retriever,
    prompt=contextualize_q_prompt
)

In [ ]:
from langchain_classic.chains import create_retrieval_chain  # as of 20251214, using classic -> this is an open issue with standard langchain library https://github.com/langchain-ai/langchain-community/issues/433
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

system_prompt_text = """
You are "Apple Sales Agent", an expert Apple product specialist.

Use a ReAct-style reasoning process INTERNALLY:
- Thought: your internal reasoning about what to do next.
- Action: the tool name and JSON arguments you want to call.
- Observation: the result returned by the tool.
- Answer: the final response you will give to the user.

The user must NEVER see Thought, Action, or Observation.
They ONLY see the final Answer.

Tools you can call:

- rag_product_search(query: str)
  Use this when you need detailed product information from the product knowledge base.
  It returns an array of chunks with product_id, title, content, and source.

When using rag_product_search:
- Craft a focused query that includes product family, use case, and key constraints.
- Read the returned chunks carefully and base your Answer only on reliable information.
- If information is missing or unclear, say you don’t know rather than inventing details.

Your goals:
1. Understand the customer's needs, constraints, and context.
2. Recommend the best Apple products, configurations, and accessories.
3. Explain trade-offs clearly and concisely.
4. Never fabricate specs, prices, or availability.
"""


system_prompt = (
system_prompt_text +
    "\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# history_aware_retriever and question_answer_chain in sequence, retaining
# intermediate outputs such as the retrieved context for convenience.
# It has input keys input and chat_history, and includes input, chat_history,
# context, and answer in its output.
rag_chain = create_retrieval_chain(history_aware_retriever,
                                   question_answer_chain)
#  contextualize_q_prompt | llm1 | retriver | qa_prpmt | llm2

In [ ]:
question = "I want to purchase an iPhone"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
print(ai_msg_1["answer"])



In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What are the models available?" # What are common ways of doing task decompsition
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

In [ ]:

chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_2["answer"]),
    ]
)
third_question = "What is the price of the oldest one"
ai_msg_3 = rag_chain.invoke({"input": third_question, "chat_history": chat_history})

print(ai_msg_3["answer"])